In [9]:
import pandas as pd
import numpy as np
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

from scipy.sparse import csr_matrix
# import matplotlib.pyplot as plt
from collections import defaultdict

from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate

In [11]:
%%time

csv_files = [
    # 'dataset/users_final_games1.csv',
    'dataset/users_final_games2.csv',
    'dataset/users_final_games3.csv'
]

dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all CSVs
data = pd.concat(dfs, ignore_index=True)

data.columns = [
    "ID",
    "PlayerID",
    "GameID",
    "GameName",
    "GameGenre",
    "RunID",
    "RunTime",
    "CategoryType",
    "PlayerCountry",
    "PlayerPronouns",
    "PlayerSignupDate"
]

# Clean up
data = data.drop_duplicates()
data = data.dropna(subset=["PlayerID", "GameID"])

# Filter out cold-start users and games
min_plays = 3

active_users = data['PlayerID'].value_counts()[lambda x: x >= min_plays].index
active_games = data['GameID'].value_counts()[lambda x: x >= min_plays].index

data = data[data['PlayerID'].isin(active_users) & data['GameID'].isin(active_games)]

# Step 2: Leave-One-Out Split
# Keep only users with at least 2 interactions
user_counts = data['PlayerID'].value_counts()
valid_users = user_counts[user_counts >= 2].index
data = data[data['PlayerID'].isin(valid_users)]

# Split: keep one item out for test per user
test_rows = (
    data.groupby('PlayerID', group_keys=False)
        .apply(lambda group: group.loc[group.sample(1, random_state=42).index])
        .reset_index(drop=True)
)

train_data = pd.merge(data, test_rows, how='outer', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

# Step 3: Convert to Surprise format
train_df = train_data[['PlayerID', 'GameID']].copy()
train_df['Rating'] = 1

test_df = test_rows[['PlayerID', 'GameID']].copy()
test_df['Rating'] = 1  # still needed for evaluation format

reader = Reader(rating_scale=(0, 1))
trainset = Dataset.load_from_df(train_df, reader).build_full_trainset()

<timed exec>:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


CPU times: user 2.06 s, sys: 126 ms, total: 2.19 s
Wall time: 2.21 s


In [12]:
%%time

# Step 4: Train the SVD model
model = SVD()
model.fit(trainset)

# Step 5: Generate top-N recommendations per user (excluding seen items)
all_game_ids = data['GameID'].unique()
top_n = defaultdict(list)

for uid in train_df['PlayerID'].unique():
    user_games = set(train_df[train_df['PlayerID'] == uid]['GameID'])
    candidate_games = [iid for iid in all_game_ids if iid not in user_games]

    predictions = [model.predict(uid, iid) for iid in candidate_games]
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

    top_n[uid] = [(pred.iid, pred.est) for pred in top_predictions]

# Step 6: Evaluate (Precision@10, Recall@10, Hit Rate@10, NDCG@10)
def evaluate_leave_one_out(top_n, test_df):
    actual = defaultdict(set)
    for _, row in test_df.iterrows():
        actual[row['PlayerID']].add(row['GameID'])

    precisions, recalls, hits, ndcgs = [], [], [], []

    for uid, recs in top_n.items():
        recommended = [iid for iid, _ in recs]
        relevant = actual.get(uid, set())

        if not relevant:
            continue

        tp = set(recommended) & relevant
        precision = len(tp) / len(recommended) if recommended else 0
        recall = len(tp) / len(relevant) if relevant else 0
        hit = 1 if tp else 0

        # NDCG
        dcg = sum(1 / np.log2(i + 2) for i, iid in enumerate(recommended) if iid in relevant)
        ideal_dcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant), len(recommended))))
        ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        hits.append(hit)
        ndcgs.append(min(ndcg, 1.0))  # clamp to 1

    return np.mean(precisions), np.mean(recalls), np.mean(hits), np.mean(ndcgs)

# Step 7: Report results
precision, recall, hit_rate, ndcg = evaluate_leave_one_out(top_n, test_df)

print("\n📊 Leave-One-Out Evaluation (SVD)")
print(f"Precision@10: {precision:.4f}")
print(f"Recall@10:    {recall:.4f}")
print(f"Hit Rate@10:  {hit_rate:.4f}")
print(f"NDCG@10:      {ndcg:.4f}")


📊 Leave-One-Out Evaluation (SVD)
Precision@10: 0.0031
Recall@10:    0.0308
Hit Rate@10:  0.0308
NDCG@10:      0.0293
CPU times: user 1min 46s, sys: 811 ms, total: 1min 47s
Wall time: 1min 48s


In [10]:
%%time

# Define hyperparameter grid
param_grid = {
    'n_factors': [50, 100],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1],
    'n_epochs': [20]
}

# Track best scores
best_precision = 0
best_hit_rate = 0
best_combined = 0
best_params_precision = None
best_params_hit = None
best_params_combined = None

results = []

for n_factors, lr, reg, n_epochs in itertools.product(
    param_grid['n_factors'],
    param_grid['lr_all'],
    param_grid['reg_all'],
    param_grid['n_epochs']
):
    print(f"\n🔧 Training with: factors={n_factors}, lr={lr}, reg={reg}, epochs={n_epochs}")
    
    model = SVD(n_factors=n_factors, lr_all=lr, reg_all=reg, n_epochs=n_epochs)
    model.fit(trainset)

    # Generate top-N recommendations
    top_n = defaultdict(list)
    for uid in train_df['PlayerID'].unique():
        user_games = set(train_df[train_df['PlayerID'] == uid]['GameID'])
        candidate_games = [iid for iid in all_game_ids if iid not in user_games]
        predictions = [model.predict(uid, iid) for iid in candidate_games]
        top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]
        top_n[uid] = [(pred.iid, pred.est) for pred in top_predictions]

    # Evaluate
    precision, recall, hit_rate, ndcg = evaluate_leave_one_out(top_n, test_df)

    print(f"Precision@10: {precision:.4f} | Hit Rate@10: {hit_rate:.4f} | NDCG@10: {ndcg:.4f}")

    # Store all results
    results.append({
        'n_factors': n_factors,
        'lr_all': lr,
        'reg_all': reg,
        'n_epochs': n_epochs,
        'precision': precision,
        'recall': recall,
        'hit_rate': hit_rate,
        'ndcg': ndcg
    })

    # Track best for precision
    if precision > best_precision:
        best_precision = precision
        best_params_precision = (n_factors, lr, reg, n_epochs)

    # Track best for hit rate
    if hit_rate > best_hit_rate:
        best_hit_rate = hit_rate
        best_params_hit = (n_factors, lr, reg, n_epochs)

    # Optional: combined score (average)
    combined_score = (precision + hit_rate) / 2
    if combined_score > best_combined:
        best_combined = combined_score
        best_params_combined = (n_factors, lr, reg, n_epochs)

# Final best scores
print(f"\n🏆 Best Precision@10: {best_precision:.4f} with params {best_params_precision}")
print(f"🏆 Best Hit Rate@10:  {best_hit_rate:.4f} with params {best_params_hit}")
print(f"🏆 Best Combined Score: {best_combined:.4f} with params {best_params_combined}")



🔧 Training with: factors=50, lr=0.002, reg=0.02, epochs=20
Precision@10: 0.0048 | Hit Rate@10: 0.0482 | NDCG@10: 0.0458

🔧 Training with: factors=50, lr=0.002, reg=0.1, epochs=20
Precision@10: 0.0047 | Hit Rate@10: 0.0472 | NDCG@10: 0.0446

🔧 Training with: factors=50, lr=0.005, reg=0.02, epochs=20
Precision@10: 0.0044 | Hit Rate@10: 0.0439 | NDCG@10: 0.0416

🔧 Training with: factors=50, lr=0.005, reg=0.1, epochs=20
Precision@10: 0.0029 | Hit Rate@10: 0.0294 | NDCG@10: 0.0274

🔧 Training with: factors=100, lr=0.002, reg=0.02, epochs=20
Precision@10: 0.0038 | Hit Rate@10: 0.0385 | NDCG@10: 0.0359

🔧 Training with: factors=100, lr=0.002, reg=0.1, epochs=20
Precision@10: 0.0038 | Hit Rate@10: 0.0380 | NDCG@10: 0.0359

🔧 Training with: factors=100, lr=0.005, reg=0.02, epochs=20
Precision@10: 0.0034 | Hit Rate@10: 0.0336 | NDCG@10: 0.0319

🔧 Training with: factors=100, lr=0.005, reg=0.1, epochs=20
Precision@10: 0.0038 | Hit Rate@10: 0.0377 | NDCG@10: 0.0353

🏆 Best Precision@10: 0.0048 wit

In [16]:
model2 = SVD(n_factors=50, lr_all=0.002, reg_all=0.02, n_epochs=20)
model2.fit(trainset)

# Step 5: Generate top-N recommendations per user (excluding seen items)
all_game_ids = data['GameID'].unique()
top_n = defaultdict(list)

for uid in train_df['PlayerID'].unique():
    user_games = set(train_df[train_df['PlayerID'] == uid]['GameID'])
    candidate_games = [iid for iid in all_game_ids if iid not in user_games]

    predictions = [model2.predict(uid, iid) for iid in candidate_games]
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

    top_n[uid] = [(pred.iid, pred.est) for pred in top_predictions]

# Step 6: Evaluate (Precision@10, Recall@10, Hit Rate@10, NDCG@10)
def evaluate_leave_one_out(top_n, test_df):
    actual = defaultdict(set)
    for _, row in test_df.iterrows():
        actual[row['PlayerID']].add(row['GameID'])

    precisions, recalls, hits, ndcgs = [], [], [], []

    for uid, recs in top_n.items():
        recommended = [iid for iid, _ in recs]
        relevant = actual.get(uid, set())

        if not relevant:
            continue

        tp = set(recommended) & relevant
        precision = len(tp) / len(recommended) if recommended else 0
        recall = len(tp) / len(relevant) if relevant else 0
        hit = 1 if tp else 0

        # NDCG
        dcg = sum(1 / np.log2(i + 2) for i, iid in enumerate(recommended) if iid in relevant)
        ideal_dcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant), len(recommended))))
        ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        hits.append(hit)
        ndcgs.append(min(ndcg, 1.0))  # clamp to 1

    return np.mean(precisions), np.mean(recalls), np.mean(hits), np.mean(ndcgs)

# Step 7: Report results
precision, recall, hit_rate, ndcg = evaluate_leave_one_out(top_n, test_df)

print("\n📊 Leave-One-Out Evaluation (SVD)")
print(f"Precision@10: {precision:.4f}")
print(f"Recall@10:    {recall:.4f}")
print(f"Hit Rate@10:  {hit_rate:.4f}")
print(f"NDCG@10:      {ndcg:.4f}")


📊 Leave-One-Out Evaluation (SVD)
Precision@10: 0.0027
Recall@10:    0.0268
Hit Rate@10:  0.0268
NDCG@10:      0.0254
